# ADIÇÃO DE BIOMASSA MICROALGAL NO PROCESSO DE ELETROCOAGULAÇÃO/ FLOTAÇÃO PARA REMOÇÃO DE ÍONS CU2+ E NI2+ DE EFLUENTES GERADOS EM INDÚSTRIAS DE GALVANOPLASTIA

### Parte integrante da tese de Doutorado na Universidade Federal do Ceará - UFC
### Autora: Liana Geisa Conrado Maia

# Teste 01

## Importando bibliotecas

In [1]:
# !pip install sklearn
import pandas as pd
import numpy as np
from pandas import read_csv
import os
from sklearn import preprocessing
from pylab import *
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (20,20)

## Carregando o dataset

In [2]:
# Carregando os dados
arquivo = 'teste_01.csv'
df = read_csv(arquivo, sep="|")

FileNotFoundError: [Errno 2] File teste_01.csv does not exist: 'teste_01.csv'

## Explorando os dados

In [ ]:
# Exibindo os dados
df.head(5)

In [ ]:
# Tipos de dados
df.dtypes

In [ ]:
# Resumo estatístico do dataframe
print(df.describe())

## Visualização gráfica dos dados

In [ ]:
cols = df.columns

In [ ]:
# Histograma
fig = plt.figure()
df[cols].hist()
plt.show()

In [ ]:
# Densidade
df[cols].plot(kind='density', 
           subplots = True, 
           layout = (6,3), 
           sharex = False)

plt.show()

In [ ]:
# Boxplot
fig = plt.figure()
df[cols].plot(kind = 'box', 
              subplots = True, 
              layout = (7,3), 
              sharey = False)
plt.tight_layout()
plt.show()

## Cálculo da Estimativa do Custo Operacional

In [ ]:
df

In [ ]:
def calculo_consumo_energia(obj):
    energia = ( (obj['Tensão']*obj['Corrente']*obj['TDH']/60)/0.002)/1000
    return energia
    
df['ConsumoEnergia_kWh/m3'] = df.apply(calculo_consumo_energia, axis=1)


def calculo_consumo_eletrodo(obj):
    eletrodo = (obj['Corrente'] * obj['TDH']*60 * 26.98)/(3*9.65*10*10*10*10)
    return eletrodo
    
df['ConsumoEletrodoAlumínio_g'] = df.apply(calculo_consumo_eletrodo, axis=1)


def custo_operacional(obj):
    energia = ( (obj['Tensão']*obj['Corrente']*obj['TDH']/60)/0.002)/1000
    eletrodo = (obj['Corrente'] * obj['TDH']*60 * 26.98)/(3*9.65*10*10*10*10)
    return 0.77283 * energia + 5.52 * 2 * eletrodo
    
df['CustoOperacional'] = df.apply(custo_operacional, axis=1)

## Correlação entre variáveis

In [ ]:
cols = ['PercentualMicroAlgas', 'TDH', 'Tensão', 'Corrente',
       'pH', 'Turbidez', 'Condutividade', 'DO680', 'Cu', 'Ni',
       '%RemoçãoTurbidez', '%RemoçãoCondutividade', '%RemoçãoDO680',
       '%RemoçãoCu', '%RemoçãoNi', 'CustoOperacional']

In [ ]:
df[cols].corr(method = 'pearson')

In [ ]:
# Matriz de Correlação
correlations = df[cols].corr()

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin = -1, vmax = 1)
fig.colorbar(cax)
ticks = np.arange(0, len(cols), 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(cols, rotation = 90)
ax.set_yticklabels(cols)
plt.show()

## Estudando o custo operacional

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="CustoOperacional", color="PercentualMicroAlgas")
fig.show()

# Estudando o Cobre (Cu)

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="%RemoçãoCu", color="PercentualMicroAlgas")
fig.show()

### Regressão linear - Cobre

In [ ]:
import numpy as np
import statsmodels.api as sm

df_filt = df

y = df_filt['%RemoçãoCu']
    
cols = [
    'TDH', 
    'Tensão', 
    'Corrente', 
]
x = df_filt[cols]

mod = sm.OLS(y, x)
res = mod.fit()
res.summary()

In [ ]:
# https://www.statsmodels.org/stable/examples/notebooks/generated/regression_plots.html

fig = sm.graphics.plot_partregress_grid(res)
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_regress_exog(res, "TDH")
fig.tight_layout(pad=1.0)

# Estudando o Níquel (Ni)

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="Ni", color='PercentualMicroAlgas')
fig.show()

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="Ni", color='PercentualMicroAlgas')
fig.show()

## Regressão Linear - Níquel

In [ ]:
import numpy as np
import statsmodels.api as sm

df_filt = df

y = df_filt['Ni']
    
cols = [
    'PercentualMicroAlgas',
    'pH',
    'TDH', 
    'Tensão', 
    'Corrente', 
]
x = df_filt[cols]

mod = sm.OLS(y, x)
res = mod.fit()
res.summary()

In [ ]:
# https://www.statsmodels.org/stable/examples/notebooks/generated/regression_plots.html

fig = sm.graphics.plot_partregress_grid(res)
fig.tight_layout(pad=1.0)

In [ ]:
fig = sm.graphics.plot_regress_exog(res, "TDH")
fig.tight_layout(pad=1.0)

## Estudando o pH

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="pH", color='PercentualMicroAlgas')
fig.show()

## Estudando a DO680

In [ ]:
import plotly.express as px

df_temp = df
fig = px.line(df_temp, x="TDH", y="DO680", color='PercentualMicroAlgas')
fig.show()